In [1]:
import pandas as pd
import sqlite3

from IPython.display import display, HTML

In [2]:
conn = sqlite3.connect("Db-IMDB-Assignment.db")


## Q1 --- List all the directors who directed a 'Comedy' movie in a leap year. (You need to check that the genre is 'Comedy’ and year is a leap year) Your query should return director name, the movie name, and the year.

In [3]:
query = """

SELECT 
    m.title movie_title,
    p.name director_name,
    m.year movie_year
FROM
    Movie m 
    JOIN
    M_Director md
    ON m.MID= md.MID
    JOIN 
    Person p
    ON md.PID = p.PID
WHERE
    ((movie_year % 4 == 0 AND movie_year % 100 != 0) 
        OR 
    (movie_year % 400 == 0))
    AND
    m.MID IN
        (
        SELECT mg.MID 
        FROM M_Genre mg
        WHERE mg.GID IN 
            (
            SELECT g.GID
             FROM Genre g
             WHERE g.Name LIKE '%Comedy%'
            )
        )
;"""

pd.read_sql_query(query, conn)

,movie_title,director_name,movie_year
0,Mastizaade,Milap Zaveri,2016
1,Harold & Kumar Go to White Castle,Danny Leiner,2004
2,Gangs of Wasseypur,Anurag Kashyap,2012
3,Around the World in 80 Days,Frank Coraci,2004
4,The Accidental Husband,Griffin Dunne,2008
...,...,...,...
227,Let's Enjoy,Siddharth Anand Kumar,2004
228,Sathyam,Amma Rajasekhar,2008
229,Tandoori Love,Oliver Paulus,2008
230,Le Halua Le,Raja Chanda,2012



## Q2 --- List the names of all the actors who played in the movie 'Anand' (1971)

In [4]:
query = """
SELECT Name 
FROM Person 
WHERE PID IN
    (
    SELECT PID 
    FROM M_Cast 
    WHERE MID IN
        (
        SELECT MID 
        FROM Movie 
        WHERE title = 'Anand'
        )
    )
;"""

pd.read_sql_query(query, conn)

,Name
0,Amitabh Bachchan
1,Rajesh Khanna
2,Sumita Sanyal
3,Ramesh Deo
4,Seema Deo
5,Asit Kumar Sen
6,Dev Kishan
7,Atam Prakash
8,Lalita Kumari
9,Savita



## Q3 --- List all the actors who acted in a film before 1970 and in a film after 1990. (That is: < 1970 and > 1990.)

In [9]:
# We are using CTE/ Temporary table for more easy undersatnding of logic
# breaking down bigger and complex problems into smaller chunks

query = """

WITH 
    Actors_before_1970 AS
    (
    SELECT DISTINCT mc.PID PID
    FROM M_Cast mc
    WHERE mc.MID IN
        (
        SELECT m.MID
        FROM Movie m
        WHERE m.year <1970
        )
    ),
        
    Actors_after_1990 AS
    (
    SELECT DISTINCT mc.PID PID
    FROM M_Cast mc
    WHERE mc.MID IN
        (
        SELECT m.MID
        FROM Movie m
        WHERE m.year >1990
        )
    )
    
SELECT 
    p.Name Actor_Name
FROM
    Actors_after_1990 a_90
JOIN
    Actors_before_1970 a_70
ON
    a_90.PID = a_70.PID
JOIN
    Person p
ON 
    a_70.PID = p.PID
    
; """

pd.read_sql_query(query, conn)

,Actor_Name
0,Rishi Kapoor
1,Amitabh Bachchan
2,Asrani
3,Zohra Sehgal
4,Parikshat Sahni
...,...
295,Poonam
296,Jamila Massey
297,K.R. Vijaya
298,Sethi



## Q4 --- List all directors who directed 10 movies or more, in descending order of the number of movies they directed. Return the directors' names and the number of movies each of them directed.

- **In the first part we will list the no. of movies every director has directed as below**

In [7]:

query = """

SELECT
    md.PID director_id,
    COUNT(md.MID) movies_count
FROM
    M_Director md
GROUP BY
    director_id
;
"""

pd.read_sql_query(query, conn)

,director_id,movies_count
0,nm0000180,1
1,nm0000187,1
2,nm0000229,1
3,nm0000269,1
4,nm0000386,1
...,...,...
1457,nm9742183,1
1458,nm9751348,1
1459,nm9793365,1
1460,nm9795684,1


- **Now using the above table we can get directors with > 10 movies and join with Person table to get the director names**

In [13]:
query = """

SELECT 
    p.Name director_name,
    a1.movies_count
FROM
    (
    SELECT 
        md.PID director_id,
        COUNT(md.MID) movies_count
    FROM 
        M_Director md
    GROUP BY 
        director_id
    HAVING 
        movies_count >= 10
    ) as a1
JOIN
    Person p
    ON
    a1.director_id = p.PID
ORDER BY
    movies_count DESC
;
"""

pd.read_sql_query(query, conn)

,director_name,movies_count
0,David Dhawan,39
1,Mahesh Bhatt,35
2,Ram Gopal Varma,30
3,Priyadarshan,30
4,Vikram Bhatt,29
5,Hrishikesh Mukherjee,27
6,Yash Chopra,21
7,Shakti Samanta,19
8,Basu Chatterjee,19
9,Subhash Ghai,18



## Q5.a --- For each year, count the number of movies in that year that had only female actors.

- **Write your query that will get movie id, and number of people for each gender**

In [16]:
query = """
WITH
    MA_Pair AS
    (
    SELECT MID,PID 
    FROM M_Cast
    GROUP BY MID,PID
    )
    
SELECT 
    map.MID, 
    p.Gender, 
    Count(*) count 
FROM
    Person p 
    JOIN 
    MA_Pair map
    ON p.PID = map.PID
GROUP BY 
    map.MID, p.Gender
;
"""

pd.read_sql_query(query, conn)

,MID,Gender,count
0,tt0021594,None,1
1,tt0021594,Female,3
2,tt0021594,Male,5
3,tt0026274,None,2
4,tt0026274,Female,11
...,...,...,...
8841,tt8932884,Female,1
8842,tt8932884,Male,2
8843,tt9007142,None,1
8844,tt9007142,Female,4


- **Write your query that will print the movies having at least one male actor using above query**

In [17]:
query = """

WITH
    MA_Pair AS
    (
    SELECT MID,PID 
    FROM M_Cast
    GROUP BY MID,PID
    )
    
SELECT 
    map.MID, 
    p.Gender, 
    Count(*) count 
FROM
    Person p 
    JOIN 
    MA_Pair map
    ON p.PID = map.PID
GROUP BY 
    map.MID, p.Gender
HAVING
    p.Gender = 'Male' 
    AND 
    count >= 1 
;
"""

pd.read_sql_query(query, conn)

,MID,Gender,count
0,tt0021594,Male,5
1,tt0026274,Male,9
2,tt0027256,Male,8
3,tt0028217,Male,7
4,tt0031580,Male,27
...,...,...,...
3464,tt8581230,Male,5
3465,tt8698956,Male,5
3466,tt8852558,Male,14
3467,tt8932884,Male,2


- **Now using the above concepts we need to solve the main problem**

In [25]:
query = """
WITH
    MA_Pair AS
    (
    SELECT MID,PID 
    FROM M_Cast
    GROUP BY MID,PID
    ),

    movies_with_male AS
    (
    SELECT DISTINCT map.MID 
    FROM 
        Person p 
        JOIN 
        MA_Pair map
        ON 
        p.PID= map.PID
    WHERE
        p.Gender IN ('Male','None')
    )

SELECT 
    m.year, 
    COUNT(*) No_of_movies_with_only_female_cast 
FROM 
    Movie m
WHERE 
    m.MID NOT IN movies_with_male 
GROUP BY 
    year
ORDER BY 
    year DESC
;
"""

pd.read_sql_query(query, conn)

,year,No_of_movies_with_only_female_cast
0,2018,1
1,2000,1
2,1999,1
3,1939,1



## Q5.b --- Now include a small change: report for each year the percentage of movies in that year with only female actors, and the total number of movies made that year. For example, one answer will be: 1990 31.81 13522 meaning that in 1990 there were 13,522 movies, and 31.81% had only female actors. You do not need to round your answer.


In [38]:
query = """
WITH
    MA_Pair AS
    (
    SELECT MID,PID 
    FROM M_Cast
    GROUP BY MID,PID
    ),

   movies_with_male AS
    (
    SELECT DISTINCT map.MID 
    FROM 
        Person p JOIN MA_Pair map
        ON p.PID= map.PID
    WHERE p.Gender IN ('Male','None')
    ),

    movies_with_female AS
    (
    SELECT 
        m.year YEAR_OF_MOVIE, 
        COUNT(*) movies_with_only_female_cast 
    FROM Movie m
    WHERE m.MID NOT IN movies_with_male 
    GROUP BY YEAR_OF_MOVIE
    ORDER BY YEAR_OF_MOVIE DESC
    ),
    
    total_movies_count AS
    (
    SELECT 
        m.year YEAR_OF_MOVIE,
        COUNT(*) total_no_of_movies
    FROM Movie m
    GROUP BY YEAR_OF_MOVIE
    )
    
SELECT 
    tmc.YEAR_OF_MOVIE,
    tmc.total_no_of_movies,
    (CAST(mwf.movies_with_only_female_cast AS float)*100 /tmc.total_no_of_movies) Percentage
FROM
    total_movies_count tmc
    JOIN 
    movies_with_female mwf
    ON mwf.YEAR_OF_MOVIE = tmc.YEAR_OF_MOVIE
    
;
"""

pd.read_sql_query(query, conn)

,YEAR_OF_MOVIE,total_no_of_movies,Percentage
0,1939,2,50.000000
1,1999,66,1.515152
2,2000,64,1.562500
3,2018,104,0.961538



## Q6 --- Find the films in order of cast size. Return the movie title and the size of the cast. By "cast size" we mean the number of distinct actors that played in that movie: if an actor played multiple roles, or if it simply occurs multiple times in casts, we still count her/him only once.

In [46]:
query = """
WITH
    Cast_Tab AS
    (
    SELECT 
        MID, 
        COUNT(DISTINCT PID) cast_size
    FROM M_Cast
    GROUP BY MID
    )
    
SELECT
    m.title movie_name,
    ct.cast_size
FROM
    Cast_Tab ct JOIN Movie m
    ON ct.MID = m.MID
ORDER BY cast_size DESC
;
"""

pd.read_sql_query(query, conn)

,movie_name,cast_size
0,Ocean's Eight,238
1,Apaharan,233
2,Gold,215
3,My Name Is Khan,213
4,Captain America: Civil War,191
...,...,...
3468,Return of Hanuman,1
3469,Chaar Sahibzaade 2: Rise of Banda Singh Bahadur,1
3470,Vaibhav Sethia: Don't,1
3471,Subah Subah,1


### Q7 --- A decade is a sequence of 10 consecutive years. 
### For example, say in your database you have movie information starting from 1931. 
### the first decade is 1931, 1932, ..., 1940,
### the second decade is 1932, 1933, ..., 1941 and so on. 
### Find the decade D with the largest number of films and the total number of films in D

</br>

- **Write a query that computes number of movies in each year**

In [49]:
query = """
SELECT 
    m.year YEAR_OF_MOVIE,
    COUNT(*) total_no_of_movies
FROM Movie m
GROUP BY YEAR_OF_MOVIE
ORDER BY YEAR_OF_MOVIE 
;
"""

pd.read_sql_query(query, conn)

,YEAR_OF_MOVIE,total_no_of_movies
0,1931,1
1,1936,3
2,1939,2
3,1941,1
4,1943,1
...,...,...
73,2014,126
74,2015,119
75,2016,129
76,2017,126


- **Write a query that will do a self join of the above table with itself such that it will join with only rows if the second tables year is <= current_year+9 and more than or equal current_year**

In [53]:
query = """
WITH 
    movies_count_table AS
    (
    SELECT 
        m.year YEAR_OF_MOVIE,
        COUNT(*) total_no_of_movies
    FROM Movie m
    GROUP BY YEAR_OF_MOVIE
    ORDER BY YEAR_OF_MOVIE 
    )
    
SELECT *
FROM 
    movies_count_table mct1,movies_count_table mct2
WHERE 
    mct2.YEAR_OF_MOVIE <= (mct1.YEAR_OF_MOVIE+9) 
    AND 
    mct2.YEAR_OF_MOVIE >= mct1.YEAR_OF_MOVIE 
;
"""

pd.read_sql_query(query, conn)

,YEAR_OF_MOVIE,total_no_of_movies,YEAR_OF_MOVIE,total_no_of_movies
0,1931,1,1931,1
1,1931,1,1936,3
2,1931,1,1939,2
3,1936,3,1936,3
4,1936,3,1939,2
...,...,...,...,...
708,2016,129,2017,126
709,2016,129,2018,104
710,2017,126,2017,126
711,2017,126,2018,104


- **Write a query that will return the decade that has maximum number of movies**

In [56]:
query = """
WITH 
    movies_count_table AS
    (
    SELECT 
        m.year YEAR_OF_MOVIE,
        COUNT(*) total_no_of_movies
    FROM Movie m
    GROUP BY YEAR_OF_MOVIE
    ORDER BY YEAR_OF_MOVIE 
    )
    
SELECT 
    mct1.YEAR_OF_MOVIE decade_start_year,
    SUM(mct2.total_no_of_movies) decade_movie_count
FROM 
    movies_count_table mct1,movies_count_table mct2
WHERE 
    mct2.YEAR_OF_MOVIE <= (mct1.YEAR_OF_MOVIE+9) 
    AND 
    mct2.YEAR_OF_MOVIE >= mct1.YEAR_OF_MOVIE 
GROUP BY
    decade_start_year
ORDER BY
    decade_movie_count DESC
LIMIT 1   
;
"""

pd.read_sql_query(query, conn)

,decade_start_year,decade_movie_count
0,2008,1203



## Q8---Find the actors that were never unemployed for more than 3 years at a stretch. (Assume that the actors remain unemployed between two consecutive movies)

In [81]:
query = """

WITH
    MA_Pair AS
    (
    SELECT MID,PID 
    FROM M_Cast mc
    GROUP BY MID,ID
    ),
    
    MY_Cast AS
    (
    SELECT 
        MAP.MID,
        M.year M_year,
        MAP.PID
    FROM 
        MA_Pair MAP JOIN Movie M ON MAP.MID = M.MID
    )

SELECT p.Name 
FROM Person p
WHERE p.PID IN 
    (
    SELECT DISTINCT MYC.PID 
    FROM MY_Cast MYC 
    WHERE MYC.PID NOT IN
        (
        SELECT DISTINCT MYC1.PID
        FROM MY_Cast MYC1
        WHERE EXISTS
            (
            SELECT *
            FROM MY_Cast MYC2
            WHERE 
                MYC2.PID = MYC1.PID
                AND
                MYC1.M_year > MYC2.M_year +3
                AND 
                NOT EXISTS
                    (
                    SELECT *
                    FROM MY_Cast MYC3
                    WHERE
                        MYC2.PID = MYC3.PID
                        AND
                        MYC3.M_year < MYC1.M_year
                        AND 
                        MYC3.M_year > MYC2.M_year
                    )
            )
        )
    )
;
"""

pd.read_sql_query(query, conn)

,Name
0,Christian Bale
1,Cate Blanchett
2,Benedict Cumberbatch
3,Naomie Harris
4,Andy Serkis
...,...
27895,Hayley Cleghorn
27896,Nirvasha Jithoo
27897,Kamal Maharshi
27898,Mohini Manik



## Q9 --- Find all the actors that made more movies with Yash Chopra than any other director.

- **Write a query that will results in number of movies actor-director worked together**

In [63]:
query = """
WITH
    MA_Pair AS
    (
    SELECT MID,PID 
    FROM M_Cast mc
    GROUP BY MID,ID
    ),

    AD_Pair AS
    (
    SELECT
        map.PID actor,
        md.PID director,
        COUNT(*) no_of_movies
    FROM
        M_Director md
        JOIN
        MA_Pair map
        ON md.MID = map.MID
    GROUP BY
        actor,director
    )
SELECT * FROM AD_Pair        
;
"""

pd.read_sql_query(query, conn)

,actor,director,no_of_movies
0,nm0000002,nm0496746,1
1,nm0000027,nm0000180,1
2,nm0000039,nm0896533,1
3,nm0000042,nm0896533,1
4,nm0000047,nm0004292,1
...,...,...,...
73403,nm9988016,nm1464314,1
73404,nm9988018,nm1464314,1
73405,nm9990703,nm4264671,1
73406,nm9990704,nm4264671,1


- **Now using the above table write the query for the main problem**

In [68]:
query = """
WITH
    MA_Pair AS
    (
    SELECT MID,PID 
    FROM M_Cast mc
    GROUP BY MID,ID
    ),

    AD_Pair AS
    (
    SELECT
        map.PID actor,
        md.PID director,
        COUNT(*) no_of_movies
    FROM
        M_Director md
        JOIN
        MA_Pair map
        ON md.MID = map.MID
    GROUP BY actor,director
    ), 
    
    max_no_movies AS
    (
    SELECT actor,max(no_of_movies) max_count
    FROM AD_Pair 
    GROUP BY actor
    )

SELECT 
    P.Name, ADP.no_of_movies
FROM 
    max_no_movies MNM, AD_Pair ADP, Person P 
WHERE 
    ADP.actor = MNM.actor
    AND 
    ADP.actor = P.PID
    AND 
    ADP.no_of_movies = MNM.max_count 
    AND 
    ADP.director IN
        (
        SELECT p.PID
        FROM Person p 
        WHERE p.Name LIKE '%Yash Chopra%'
        )
ORDER BY
    MNM.max_count DESC    
        
;
"""

pd.read_sql_query(query, conn)

,Name,no_of_movies
0,Jagdish Raj,11
1,Manmohan Krishna,10
2,Iftekhar,9
3,Shashi Kapoor,7
4,Rakhee Gulzar,5
...,...,...
240,Abbie Murphy,1
241,Richard Broom,1
242,Vinita Sharma,1
243,Sean Moon,1



## Q10--- The Shahrukh number of an actor is the length of the shortest path between the actor and Shahrukh Khan in the "co-acting" graph. That is, Shahrukh Khan has Shahrukh number 0; all actors who acted in the same film as Shahrukh have Shahrukh number 1; all actors who acted in the same film as some actor with Shahrukh number 1 have Shahrukh number 2, etc. Return all actors whose Shahrukh number is 2.

- **selecting actors who acted with srk (S1)**

In [71]:
query = """
WITH 
    S1_ACTORS AS
    (
    SELECT DISTINCT MC.PID S1_PID
    FROM M_Cast MC 
    WHERE MC.PID NOT IN 
        (
        SELECT PID FROM Person WHERE Name like "%Shah rukh khan%"
        )
        AND 
        MC.MID IN
        (
        SELECT DISTINCT MC1.MID 
        FROM M_Cast MC1 
        WHERE MC1.PID IN
            (
            SELECT PID FROM Person WHERE Name like "%Shah rukh khan%"
            )
        )
    )
    
SELECT * FROM S1_ACTORS
;
"""

pd.read_sql_query(query, conn)

,S1_PID
0,nm0004418
1,nm1995953
2,nm2778261
3,nm0631373
4,nm0241935
...,...
2377,nm4173451
2378,nm7620177
2379,nm3093045
2380,nm0451154


### Now using the following logic we can solve the problem
- **selecting all movies where S1 actors acted, this forms S2 movies list**
- **selecting all actors who acted in S2 movies, this gives us S2 actors along with S1 actors**
- **removing S1 actors from the combined list of S1 & S2 actors, so that we get only S2 actors**

In [75]:
query = """
WITH 
    S1_ACTORS AS
    (
    SELECT DISTINCT MC.PID S1_PID
    FROM M_Cast MC 
    WHERE MC.PID NOT IN 
        (
        SELECT PID FROM Person WHERE Name like "%Shah rukh khan%"
        )
        AND 
        MC.MID IN
        (
        SELECT DISTINCT MC1.MID 
        FROM M_Cast MC1 
        WHERE MC1.PID IN
            (
            SELECT PID FROM Person WHERE Name like "%Shah rukh khan%"
            )
        )
    ),
    
    S2_S1_ACTORS AS
    (
    SELECT DISTINCT MC.PID S2_S1_PID 
    FROM M_Cast MC
    WHERE MC.MID IN
        (
        SELECT DISTINCT MC1.MID 
        FROM M_Cast MC1
        WHERE MC1.PID IN
            (
            SELECT S1A.S1_PID FROM S1_ACTORS S1A
            )
        )
    ),
    
    S2_ACTORS AS 
    (
    SELECT S2S1A.S2_S1_PID as S2_PID
    FROM S2_S1_ACTORS S2S1A 
    WHERE 
        S2S1A.S2_S1_PID NOT IN 
            (
            SELECT S1_PID FROM S1_ACTORS
            )
        AND S2_S1_PID NOT IN
            (
            SELECT PID FROM Person WHERE Name like "%Shah rukh khan%"
            )
    )
    
SELECT Name 
FROM person 
WHERE PID IN
    (
    SELECT * FROM S2_ACTORS
    )
;
"""

pd.read_sql_query(query, conn)

,Name
0,Freida Pinto
1,Rohan Chand
2,Damian Young
3,Waris Ahluwalia
4,Caroline Christl Long
...,...
25693,Minoo Mehtab
25694,Hayley Cleghorn
25695,Nirvasha Jithoo
25696,Kamal Maharshi
